In [1]:
# This is necessary to recognize the modules
import os
import sys
from decimal import Decimal

import pandas as pd

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

In [2]:
from data_sources.clob import CLOBDataSource

# Get trading rules and candles
clob = CLOBDataSource()

In [3]:
# Constants
CONNECTOR_NAME = "binance_perpetual"
INTERVAL = "1m"
DAYS = 20

# Features configuration
VOLATILITY_WINDOW = 100
BOLLINGER_LENGTH = 200
BOLLINGER_STD = 2.0

# Download data
- Get trading rules
- Get candles for the last x days

In [26]:
import asyncio

trading_rules = await clob.get_trading_rules(CONNECTOR_NAME)
trading_pairs = trading_rules.filter_by_base_asset("DOGE")\
    .filter_by_quote_asset("USDT")\
    .filter_by_min_notional_size(Decimal("5"))\
    .get_all_trading_pairs()

In [28]:
# print(trading_rules.get_all_trading_pairs())
# print(trading_pairs)

In [29]:
tasks = [clob.get_candles_last_days(
    connector_name=CONNECTOR_NAME,
    trading_pair=trading_pair,
    interval=INTERVAL,
    days=DAYS,
) for trading_pair in trading_pairs[:5]]
candles = await asyncio.gather(*tasks)
candles = {trading_pair: candle for trading_pair, candle in zip(trading_pairs, candles)}

2024-08-24 11:34:12,329 - asyncio - ERROR - Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1488d1b40>
2024-08-24 11:34:12,329 - asyncio - ERROR - Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x1488b8220>, 201416.722850458)]']
connector: <aiohttp.connector.TCPConnector object at 0x1488d22f0>


In [35]:
clob.dump_candles_cache(os.path.join(root_path, "data"))

In [36]:
clob.candles_cache.keys()


dict_keys([('binance_perpetual', 'DOGE-USDT', '1m')])

In [37]:
os.listdir(os.path.join(os.path.join(root_path, "data"), "candles"))


['.gitignore', 'binance_perpetual|DOGE-USDT|1m.csv']

In [38]:
clob.load_candles_cache(os.path.join(root_path, "data"))


In [39]:
candles.keys()


dict_keys(['DOGE-USDT'])

In [42]:
trading_pair = list(candles.keys())[0]
candles[trading_pair].plot(type="candles")

In [41]:
candles[trading_pair].plot(type="returns")